# 문8) 배열을 이용한 연결 리스트란? 포인터 연결리스트와 비교

**각 노드를 배열안의 원소로 저장하고, 이 원소들을 연결리스트로 구현하는 방법**  
  
데이터의 개수가 크게 변하지 않거나  
데이터 최대 개수를 예측할 수 있는 경우엔
    
    이 방법이 더 효율적이다

포인터 연결리스트는 

    노드를 삽입/삭제할 때마다 
    내부에서 노드용 인스턴스를 생성/소멸시켜 
    이때 메모리를 확보/해제 하는데 비용이 생긴다

# 문9) 프리리스트란?

### 배열에서 물리적인 위치 관계와 연결리스트이 논리적인 순서 관계가 일치하는 것은 아니다.
즉 연결리스트상에서 n번째에 위치하는 노드가, 배열 인덱스 n의 원소에 저장되는 건 아니다

    노드가 논리적으로 연결되어 있는 순서와, 노드가 실제로 저장된 위치를 구별하기 위해 
    배열 인덱스 n에 노드가 저장되는 곳을 **n번째 레코드** 라고 부르자
    
    만약 노드 삭제를 여러번 하게 되면
    
    배열 안에 빈 레코드가 많이 생긴다.
    
### 프리 리스트 free list는 빈 레코드들이 연결되어 있는 순서를 나타낸다
### i.e.프리리스트는, 
### 배열 n에서 연결리스트의 노드가 사용하지 않는 인덱스를 커서로 사용한다.
책 352p 그림 8-19

    프리 리스트를 사용하면, 삭제후 비어 있는 배열의 문제를 해결할 수 있다.

**연결리스트의 각 노드는**  
**연결리스트 상에서 다음 노드가 위치하는 배열의 인덱스를 참조하는 next 필드뿐만 아니라**  
**프리리스트 상에서 다음 빈 레코드를 참조하는 **dnext** 필드도** 가지고 있다  

### 연결리스트에서 어떤 노드가 처음 삭제되면, 그 노드는 프리리스트의 머리 노드가 된다.
### 연결리스트에서 또 다른 어떤 노드가 삭제되면, 그 노드는 프리리스트의 새로운 머리노드가 된다.
### 반대로 연결리스트에 어떤 노드가 추가되면 프리리스트의 머리노드가 삭제된다

# 문10) 커서란?

**커서 : 다음 노드가 저장된 배열의 인덱스를 나타내는 int**  

    연결리스트 상에서 다음 노드가 위치하는 배열의 인덱스를 참조하는 필드 self.next가 참조하는 값

    머리노드의 커서가 1이라는 건
        배열의 1번 인덱스에 머리노드의 다음 노드가 있다는 것.

    어떤 노드의 커서가 -1이라는 건
        그 어떤 노드가 꼬리노드라는 것.

커서 연결리스트에서는

    머리노드 앞에 노드를 추가하게 되면

    self.head의 커서를 새로 추가되는 노드가 위치한 배열의 인덱스로 바꾸고

    새로 추가되는 노드의 커서를 기존 머리노드가 위치한 배열의 인덱스로 하면 된다.

# 문11) 배열을 이용한 연결리스트 with 프리리스트 구현방법
노드클래스, 연결리스트 클래스, __len__, get_insert_index(), delete_index(), search(), __contains__, add_first(), add_last(), remove_first(), remove_last(), remove(), remove_current_node(), clear(), next(), print_current_node(), print(),m dump(), __iter__, 이터레이터 클래스

In [15]:
from __future__ import annotations
from typing import Any, Type

Null = -1


class Node:
    
    
    def __init__(self, data = Null, next = Null, dnext = Null):
        self.data = data
        self.next = next # 연결리스트 상에서 다음 노드가 위치하는 배열의 인덱스를 참조하는 필드
        self.dnext = dnext  # 프리리스트 상에서 다음 노드가 위치하는 배열의 인덱스(빈레코드)를 참조하는 필드
    
    
class ArrayLinkedList:
    
    
    def __init__(self, capacity: int):
        
        self.head = Null       # self.head : 머리노드가 저장된 배열의 인덱스를 참조
        self.current = Null    # self.current : 현재 주목하는 노드가 저장된 배열의 인덱스를 참조
        self.max = Null        # 배열의 맨 끝 쪽에 저장되는 노드(꼬리 레코드)의 레코드 번호(인덱스) *꼬리노드의 인덱스가 아니다
        self.deleted = Null    # self.deleted : 프리 리스트의 머리 노드를 참조 
        self.capacity = capacity
        self.n = [Node()] * self.capacity # 연결리스트가 저장되는 배열 n
        self.no = 0
        
    def __len__(self) -> int:
        
        return self.no
    
    def get_insert_index(self):  # 연결리스트에 노드를 추가할 때, 프리리스트의 머리노드 레코드를 가져오는 함수
        
        if self.deleted == Null: # 프리리스트가 비었다면, i.e. 삭제된 레코드가 하나도 없었다면 or 배열이 꽉 찼다면
            if self.max + 1 < self.capacity:   # max를 1 증가시켰을때, 처음에 설정한 배열 크기보다 작으면 
                self.max += 1                  # max +1 하고 self.max값 반환
                return self.max                # 이런식으로 초기 빈 배열에 노드를 계속 추가해나가면서 삭제없이 배열을 꽉 채우거나
            else:                              # 중간에 삭제와 추가를 반복해서 결국 빈 레코드 없이 배열이 꽉 차도
                                               # self.max 값은 배열의 마지막 인덱스가 된다. 
                                               # 따라서 이때 +1을 하면 self.capacity와 값이 같다. 
                return Null      # i.e.배열 크기 초과
        
        else:                    # 프리리스트가 비어있지 않다면
            rec = self.deleted   # rec: 프리리스트의 머리노드가 위치해있는 배열의 인덱스를 참조하는 self.deleted를 참조
            self.deleted = self.n[rec].dnext  # self.deleted는 프리리스트 머리노드의 다음노드가 위치한 배열의 인덱스를 참조
                                              # 즉 프리리스트의 머리노드의 다음노드를 새로운 머리노드로 설정
            return rec           # 기존 프리리스트 머리노드가 위치해있는 배열의 인덱스를 반환
    
    def delete_index(self, idx: int) -> None:    # 삭제할 노드를 프리리스트의 머리노드로 등록하는 함수
        
        if self.deleted == Null:      # 프리리스트가 비었다면
            self.deleted = idx        # self.deleted 필드는 삭제할 노드의 레코드 번호 idx를 참조
                                      # 즉 삭제할 노드를 프리리스트의 머리노드로 등록 
            self.n[idx].dnext = Null  # 새로 등록된 프리리스트의 머리노드의 dnext 필드는 -1을 참조, 프리리스트에 노드는 하나이므로.
            
        else:                     # 프리리스트가 비어있지 않다면
            rec = self.deleted    # rec : 프리리스트 머리노드를 참조하는 self.deleted를 참조
            self.deleted = idx    # self.deleted는 삭제할 노드의 레코드 번호 idx를 참조, 
                                   # 즉 삭제할 노드가 새로운 프리리스트의 머리노드가 된다.
            self.n[idx].dnext = rec # 새로운 프리리스트 머리노드의 다음 노드가 기존 프리리스트 머리노드가 되도록 한다.
            
            # !!!!!!!!!!!!!! 즉 연결리스트에서 어떤 노드가 삭제되면, 그 노드는 프리리스트의 머리 노드가 되는 것이다.
            
    def search(self, data: Any) -> int:  # 검색하는 값과 일치하는 데이터를 가진 노드의 연결리스트 상 순번을 반환
        cnt = 0
        ptr = self.head
        while ptr != Null:  # 연결리스트 끝까지 검색
            if self.n[ptr].data == data:
                self.current = ptr
                return cnt
            cnt += 1
            ptr = self.n[ptr].next
        return Null # 검색 실패
    
    def __contains__(self, data: Any) -> bool:
        
        return self.search(data) >= 0
    
    def add_first(self, data: Any):
        print(f'rec 참조전 self.head : {self.head}')
        print(f'rec 참조전 self.current : {self.current}')
        ptr = self.head                 # 비어 있는 연결리스트였다면 ptr은 -1 참조
        print(f'ptr : {ptr}')
        rec = self.get_insert_index()   # 비어 있는 연결리스트였다면 rec는 0 참조. 왜냐면 self.max 는 -1이었고 여기에 +1
        print(f'rec : {rec}')
        if rec != Null:  # 배열 크기를 초과하지 않았다면
            self.head = self.current = rec
            print(f'rec 참조후 self.head : {self.head}')
            print(f'rec 참조후 self.current : {self.current}')
            self.n[self.head] = Node(data, ptr)  # 비어있는 연결리스트였다면 배열 n의 0번째 인덱스, i.e. 0번 레코드에 노드 추가. 
                                                 # 비어있는 연결리스트였다면 새로 추가된 노드의 self.next 는 ptr 즉 -1을 참조
            self.no += 1

### 커서 연결리스트에서 
#### ArrayLinkedList 클래스 생성자의 head, current, max, deleted 그리고 ptr 같은 변수들은 
#### 정수 즉 배열 n의 인덱스(레코드)를 참조한다.
    해당 인덱스로 접근되는 곳에 연결리스트의 노드가 저장된다
## 이 점이 포인터 연결리스트와 큰 차이
### 포인터 연결리스트의 생성자의 head, current 그리고 ptr 같은 변수들은 
### 노드의 인스턴스를 참조한다.

# 커서 연결리스트에서 add_first()의 의미는
연결리스트 상에서 머리쪽에 노드를 추가한다는 뜻이다.  
    배열상에서의 위치는 관련 없다
## 레코드, 즉 배열의 인덱스 값의 크기는 무시하고 
## head와 current 그리고 head의 next 값을 봐야 알 수 있다.
### 빈 배열에서 시작하면, 배열의 인덱스가 +1씩 늘어나면서 노드를 순차적으로 저장하므로
### 최근에 추가된 노드의 레코드 값은 크지만, i.e. head와 current 값도 크지만
### 연결리스트를 머리부터 꼬리까지 따라가면 next 값은 거꾸로 줄어든다.

In [16]:
ALL = ArrayLinkedList(10)
ALL.add_first(40)

rec 참조전 self.head : -1
rec 참조전 self.current : -1
ptr : -1
rec : 0
rec 참조후 self.head : 0
rec 참조후 self.current : 0


In [17]:
print(f'ALL.head : {ALL.head}')
print(f'ALL.current : {ALL.current}')
print(f'ALL.n[0].data : {ALL.n[0].data}')
print(f'ALL.n[0].next : {ALL.n[0].next}')  # ptr과 next의 값이 같다. 

ALL.head : 0
ALL.current : 0
ALL.n[0].data : 40
ALL.n[0].next : -1


In [18]:
ALL.add_first(30)

rec 참조전 self.head : 0
rec 참조전 self.current : 0
ptr : 0
rec : 1
rec 참조후 self.head : 1
rec 참조후 self.current : 1


In [19]:
print(f'ALL.head : {ALL.head}')
print(f'ALL.current : {ALL.current}')
print(f'ALL.n[0].data : {ALL.n[0].data}')
print(f'ALL.n[0].next : {ALL.n[0].next}')
print(f'ALL.n[1].data : {ALL.n[1].data}')
print(f'ALL.n[1].next : {ALL.n[1].next}')

ALL.head : 1
ALL.current : 1
ALL.n[0].data : 40
ALL.n[0].next : -1
ALL.n[1].data : 30
ALL.n[1].next : 0


In [20]:
ALL.add_first(20)

rec 참조전 self.head : 1
rec 참조전 self.current : 1
ptr : 1
rec : 2
rec 참조후 self.head : 2
rec 참조후 self.current : 2


In [21]:
print(f'ALL.head : {ALL.head}')
print(f'ALL.current : {ALL.current}')
print(f'ALL.n[0].data : {ALL.n[0].data}')
print(f'ALL.n[0].next : {ALL.n[0].next}')
print(f'ALL.n[1].data : {ALL.n[1].data}')
print(f'ALL.n[1].next : {ALL.n[1].next}')
print(f'ALL.n[2].data : {ALL.n[2].data}')
print(f'ALL.n[2].next : {ALL.n[2].next}')

ALL.head : 2
ALL.current : 2
ALL.n[0].data : 40
ALL.n[0].next : -1
ALL.n[1].data : 30
ALL.n[1].next : 0
ALL.n[2].data : 20
ALL.n[2].next : 1


In [22]:
ALL.add_first(10)

rec 참조전 self.head : 2
rec 참조전 self.current : 2
ptr : 2
rec : 3
rec 참조후 self.head : 3
rec 참조후 self.current : 3


In [23]:
print(f'ALL.head : {ALL.head}')
print(f'ALL.current : {ALL.current}')
print(f'ALL.n[0].data : {ALL.n[0].data}')
print(f'ALL.n[0].next : {ALL.n[0].next}')
print(f'ALL.n[1].data : {ALL.n[1].data}')
print(f'ALL.n[1].next : {ALL.n[1].next}')
print(f'ALL.n[2].data : {ALL.n[2].data}')
print(f'ALL.n[2].next : {ALL.n[2].next}')
print(f'ALL.n[3].data : {ALL.n[3].data}')
print(f'ALL.n[3].next : {ALL.n[3].next}')

ALL.head : 3
ALL.current : 3
ALL.n[0].data : 40
ALL.n[0].next : -1
ALL.n[1].data : 30
ALL.n[1].next : 0
ALL.n[2].data : 20
ALL.n[2].next : 1
ALL.n[3].data : 10
ALL.n[3].next : 2


In [118]:
from __future__ import annotations
from typing import Any, Type

Null = -1


class Node:
    
    
    def __init__(self, data = Null, next = Null, dnext = Null):
        self.data = data
        self.next = next # 연결리스트 상에서 다음 노드가 위치하는 배열의 인덱스를 참조하는 필드
        self.dnext = dnext  # 프리리스트 상에서 다음 노드가 위치하는 배열의 인덱스(빈레코드)를 참조하는 필드
    
    
class ArrayLinkedList:
    
    
    def __init__(self, capacity: int):
        
        self.head = Null       # self.head : 머리노드가 저장된 배열의 인덱스를 참조
        self.current = Null    # self.current : 현재 주목하는 노드가 저장된 배열의 인덱스를 참조
        self.max = Null        # 배열의 맨 끝 쪽에 저장되는 노드(꼬리 레코드)의 레코드 번호(인덱스) *꼬리노드의 인덱스가 아니다
        self.deleted = Null    # self.deleted : 프리 리스트의 머리 노드를 참조 
        self.capacity = capacity
        self.n = [Node()] * self.capacity # 연결리스트가 저장되는 배열 n
        self.no = 0
        
    def __len__(self) -> int:
        
        return self.no
    
    def get_insert_index(self):  # 연결리스트에 노드를 추가할 때, 프리리스트의 머리노드 레코드를 가져오는 함수
        
        if self.deleted == Null: # 프리리스트가 비었다면, i.e. 삭제된 레코드가 하나도 없었다면 or 배열이 꽉 찼다면
            if self.max + 1 < self.capacity:   # max를 1 증가시켰을때, 처음에 설정한 배열 크기보다 작으면 
                self.max += 1                  # max +1 하고 self.max값 반환
                return self.max                # 이런식으로 초기 빈 배열에 노드를 계속 추가해나가면서 삭제없이 배열을 꽉 채우거나
            else:                              # 중간에 삭제와 추가를 반복해서 결국 빈 레코드 없이 배열이 꽉 차도
                                               # self.max 값은 배열의 마지막 인덱스가 된다. 
                                               # 따라서 이때 +1을 하면 self.capacity와 값이 같다. 
                return Null      # i.e.배열 크기 초과
        
        else:                    # 프리리스트가 비어있지 않다면
            rec = self.deleted   # rec: 프리리스트의 머리노드가 위치해있는 배열의 인덱스를 참조하는 self.deleted를 참조
            self.deleted = self.n[rec].dnext  # self.deleted는 프리리스트 머리노드의 다음노드가 위치한 배열의 인덱스를 참조
                                              # 즉 프리리스트의 머리노드의 다음노드를 새로운 머리노드로 설정
            return rec           # 기존 프리리스트 머리노드가 위치해있는 배열의 인덱스를 반환
    
    def delete_index(self, idx: int) -> None:    # 삭제할 노드를 프리리스트의 머리노드로 등록하는 함수
        
        if self.deleted == Null:      # 프리리스트가 비었다면
            self.deleted = idx        # self.deleted 필드는 삭제할 노드의 레코드 번호 idx를 참조
                                      # 즉 삭제할 노드를 프리리스트의 머리노드로 등록 
            self.n[idx].dnext = Null  # 새로 등록된 프리리스트의 머리노드의 dnext 필드는 -1을 참조, 프리리스트에 노드는 하나이므로.
            
        else:                     # 프리리스트가 비어있지 않다면
            rec = self.deleted    # rec : 프리리스트 머리노드를 참조하는 self.deleted를 참조
            self.deleted = idx    # self.deleted는 삭제할 노드의 레코드 번호 idx를 참조, 
                                   # 즉 삭제할 노드가 새로운 프리리스트의 머리노드가 된다.
            self.n[idx].dnext = rec # 새로운 프리리스트 머리노드의 다음 노드가 기존 프리리스트 머리노드가 되도록 한다.
            
            # !!!!!!!!!!!!!! 즉 연결리스트에서 어떤 노드가 삭제되면, 그 노드는 프리리스트의 머리 노드가 되는 것이다.
            
    def search(self, data: Any) -> int:  # 검색하는 값과 일치하는 데이터를 가진 노드의 연결리스트 상 순번을 반환
        cnt = 0
        ptr = self.head
        while ptr != Null:  # 연결리스트 끝까지 검색
            if self.n[ptr].data == data:
                self.current = ptr
                return cnt
            cnt += 1
            ptr = self.n[ptr].next
        return Null # 검색 실패
    
    def __contains__(self, data: Any) -> bool:
        
        return self.search(data) >= 0
    
    def add_first(self, data: Any):
        
        ptr = self.head                 # 비어 있는 연결리스트였다면 ptr은 -1 참조
        rec = self.get_insert_index()   # 비어 있는 연결리스트였다면 rec는 0 참조. 왜냐면 self.max 는 -1이었고 여기에 +1
        if rec != Null:  # 배열 크기를 초과하지 않았다면
            self.head = self.current = rec
            self.n[self.head] = Node(data, ptr)  # 비어있는 연결리스트였다면 배열 n의 0번째 인덱스, i.e. 0번 레코드에 노드 추가. 
                                                 # 비어있는 연결리스트였다면 새로 추가된 노드의 self.next 는 ptr 즉 -1을 참조
            self.no += 1
            
    def add_last(self, data: Any) -> None:
        
        if self.head == Null:  # 비어 있는 연결리스트면
            self.add_first(data)
        else:
            ptr = self.head   # 연결리스트에 노드가 하나 있다면 ptr은 0을 참조, 즉 self.head = 0
            while self.n[ptr].next != Null: # 연결리스트 끝까지 가서
                ptr = self.n[ptr].next
            rec = self.get_insert_index()   # 추가될 노드가 저장될 인덱스를 rec가 참조
                                            # 아직 프리리스트가 빈 상태이므로 self.max = 1, 즉 rec = 1
            if rec != Null:  # 배열 크기를 초과하지 않았다면
                self.n[ptr].next = self.current = rec  # 꼬리노드의 다음노드 커서가 새로 추가될 노드가 저장될 배열의 인덱스를 참조
                self.n[rec] = Node(data)        # 새로 추가될 노드가 저장될 배열의 인덱스 rec 가 가리키는 곳에 새 노드 저장
                self.no += 1
                
    def remove_first(self) -> None:
        
        if self.head != Null:   # 연결리스트가 비어있지 않으면
            ptr = self.n[self.head].next  # ptr은 연결리스트의 머리노드의 다음 노드를 가리키는 인덱스를 참조
            self.delete_index(self.head)  # 연결리스트의 머리노드를 프리리스트의 머리노드로 등록
            self.head = self.current = ptr # 연결리스트의 머리노드 삭제
            self.no -= 1
            
    def remove_last(self) -> None:
        
        if self.head != Null:  # 연결리스트가 비어있지 않으면
            if self.n[self.head].next == Null: # 연결리스트에 노드가 하나뿐이면
                self.remove_first()
            else:              # 연결리스트가 비어있지 않으면
                ptr = self.head
                pre = self.head
                
                while self.n[ptr].next != Null:  # 연결리스트 끝까지 가서
                    pre = ptr                    # pre 는 꼬리노드 직전 노드를 참조
                    ptr = self.n[ptr].next
                self.n[pre].next = Null          # 꼬리노드 삭제
                self.delete_index(ptr)           # 삭제된 연결리스트의 꼬리노드를 프리리스트의 머리노드로 등록
                self.current = pre
                self.no -= 1
                
    def remove(self, p: int) -> None:
        
        if self.head != Null:   # 연결리스트가 비어있지 않으면
            if p == self.head:  # 지우려는 노드가 연결리스트의 머리노드면
                self.remove_first()
            else:
                ptr = self.head # 아니면 ptr은 연결리스트의 머리노드 참조
                
                while self.n[ptr].next != p: # "어떤 노드"의 다음 노드가 지우려는 노드가 될 때까지 반복                                         # p가 참조하는 인덱스면
                                             # 지우려는 노드가 current 노드면, "어떤 노드"는 연결리스트상 current노드의 직전 노드
                    ptr = self.n[ptr].next
                    if ptr == Null:          # 만약 "어떤 노드"의 다음노드가 -1이면 "어떤 노드"는 꼬리노드란 소리이므로
                        return               # 검색 종료
                                             
                self.delete_index(p)               # 지우려는 노드를 프리리스트의 머리노드로 등록
                self.n[ptr].next = self.n[p].next  # "어떤 노드"의 다음 노드가 지우려는 노드의 다음노드가 되도록 한다. 
                self.current = ptr                 # self.current는 "어떤 노드"를 참조하도록 한다. 
                self.no -= 1
                
    def remove_current_node(self) -> None:
        
        self.remove(self.current)
        
    def clear(self) -> None:
        
        while self.head != Null:  # 연결리스트 전체가 빌 때까지
            self.remove_first()
        self.current = Null
        
    def next(self) -> bool:
        
        if self.current == Null or self.n[self.current].next == Null: # 연결리스트가 비어 있거나 노드가 하나밖에 없다면
            return False
        self.current = self.n[self.current].next
        return True
    
    def print_current_node(self) -> None:
        
        if self.current == Null:
            print("연결리스트가 비어있습니다")
        else: print(self.n[self.curent].data)
        
    def print(self) -> None:
        
        ptr = self.head
        
        while ptr != Null:
            print(self.n[ptr].data)
            ptr = self.n[ptr].next
            
    def dump(self) -> None:   # dump()는 배열상 노드가 저장된 순서대로 연결리스트를 출력
        print(f'머리노드의 레코드 : {self.head}')
        for i, n in enumerate(self.n):
            print(f'{i}번째 레코드의 [data, next, dnext] : [{n.data}, {n.next}, {n.dnext}]')
            
    def __iter__(self) -> ArrayLinkedListIterator:
        
        return ArrayLinkedListIterator(self.n, self.head)
    
    
class ArrayLinkedListIterator:
    
    
    def __init__(self, n: int, head: int):
        
        self.n = n
        self.current = head
        
    def __iter__(self) -> ArrayLinkedListIterator:
        
        return self
    
    def __next__(self) -> Any:
        if self.current == Null:
            raise StopIteration
        else:
            data = self.n[self.current].data
            self.current = self.n[self.current].next
            return data

In [122]:
ALL = ArrayLinkedList(10)
ALL.add_first(40); print()
ALL.add_first(30); print()
ALL.add_first(20); print()
ALL.add_first(10); print()
ALL.add_last(50)

In [123]:
print(f'ALL.head : {ALL.head}')
print(f'ALL.current : {ALL.current}')
print(f'ALL.n[0].data : {ALL.n[0].data}')
print(f'ALL.n[0].next : {ALL.n[0].next}')
print(f'ALL.n[1].data : {ALL.n[1].data}')
print(f'ALL.n[1].next : {ALL.n[1].next}')
print(f'ALL.n[2].data : {ALL.n[2].data}')
print(f'ALL.n[2].next : {ALL.n[2].next}')
print(f'ALL.n[3].data : {ALL.n[3].data}')
print(f'ALL.n[3].next : {ALL.n[3].next}')
print(f'ALL.n[4].data : {ALL.n[4].data}')
print(f'ALL.n[4].next : {ALL.n[4].next}')

ALL.head : 3
ALL.current : 4
ALL.n[0].data : 40
ALL.n[0].next : 4
ALL.n[1].data : 30
ALL.n[1].next : 0
ALL.n[2].data : 20
ALL.n[2].next : 1
ALL.n[3].data : 10
ALL.n[3].next : 2
ALL.n[4].data : 50
ALL.n[4].next : -1


In [124]:
ALL.dump()

머리노드의 레코드 : 3
0번째 레코드의 [data, next, dnext] : [40, 4, -1]
1번째 레코드의 [data, next, dnext] : [30, 0, -1]
2번째 레코드의 [data, next, dnext] : [20, 1, -1]
3번째 레코드의 [data, next, dnext] : [10, 2, -1]
4번째 레코드의 [data, next, dnext] : [50, -1, -1]
5번째 레코드의 [data, next, dnext] : [-1, -1, -1]
6번째 레코드의 [data, next, dnext] : [-1, -1, -1]
7번째 레코드의 [data, next, dnext] : [-1, -1, -1]
8번째 레코드의 [data, next, dnext] : [-1, -1, -1]
9번째 레코드의 [data, next, dnext] : [-1, -1, -1]


In [125]:
ALL.remove(ALL.current)
print(f'ALL.head : {ALL.head}')
print(f'ALL.current : {ALL.current}')
print(f'ALL.n[0].data : {ALL.n[0].data}')
print(f'ALL.n[0].next : {ALL.n[0].next}')
print(f'ALL.n[1].data : {ALL.n[1].data}')
print(f'ALL.n[1].next : {ALL.n[1].next}')
print(f'ALL.n[2].data : {ALL.n[2].data}')
print(f'ALL.n[2].next : {ALL.n[2].next}')
print(f'ALL.n[3].data : {ALL.n[3].data}')
print(f'ALL.n[3].next : {ALL.n[3].next}')
print(f'ALL.n[4].data : {ALL.n[4].data}')
print(f'ALL.n[4].next : {ALL.n[4].next}')

ALL.head : 3
ALL.current : 0
ALL.n[0].data : 40
ALL.n[0].next : -1
ALL.n[1].data : 30
ALL.n[1].next : 0
ALL.n[2].data : 20
ALL.n[2].next : 1
ALL.n[3].data : 10
ALL.n[3].next : 2
ALL.n[4].data : 50
ALL.n[4].next : -1


In [126]:
ALL.dump()

머리노드의 레코드 : 3
0번째 레코드의 [data, next, dnext] : [40, -1, -1]
1번째 레코드의 [data, next, dnext] : [30, 0, -1]
2번째 레코드의 [data, next, dnext] : [20, 1, -1]
3번째 레코드의 [data, next, dnext] : [10, 2, -1]
4번째 레코드의 [data, next, dnext] : [50, -1, -1]
5번째 레코드의 [data, next, dnext] : [-1, -1, -1]
6번째 레코드의 [data, next, dnext] : [-1, -1, -1]
7번째 레코드의 [data, next, dnext] : [-1, -1, -1]
8번째 레코드의 [data, next, dnext] : [-1, -1, -1]
9번째 레코드의 [data, next, dnext] : [-1, -1, -1]


연결리스트의 current가 꼬리노드를 보고 있을 때,

    def remove() 함수로 current 노드를 삭제하게 되면 
    self.n[ptr].next = self.n[p].next 에 의해, 새로운 current 노드의 커서는 -1을 참조한다. 
    왜냐면 삭제된 current 노드는 꼬리노드였으므로 그 커서값은 -1이니까

In [127]:
ALL.search(20)
ALL.remove(ALL.current)
print(f'ALL.head : {ALL.head}')
print(f'ALL.current : {ALL.current}')
print(f'ALL.n[0].data : {ALL.n[0].data}')
print(f'ALL.n[0].next : {ALL.n[0].next}')
print(f'ALL.n[1].data : {ALL.n[1].data}')
print(f'ALL.n[1].next : {ALL.n[1].next}')
print(f'ALL.n[2].data : {ALL.n[2].data}')
print(f'ALL.n[2].next : {ALL.n[2].next}')
print(f'ALL.n[3].data : {ALL.n[3].data}')
print(f'ALL.n[3].next : {ALL.n[3].next}')
print(f'ALL.n[4].data : {ALL.n[4].data}')
print(f'ALL.n[4].next : {ALL.n[4].next}')

ALL.head : 3
ALL.current : 3
ALL.n[0].data : 40
ALL.n[0].next : -1
ALL.n[1].data : 30
ALL.n[1].next : 0
ALL.n[2].data : 20
ALL.n[2].next : 1
ALL.n[3].data : 10
ALL.n[3].next : 1
ALL.n[4].data : 50
ALL.n[4].next : -1


In [128]:
ALL.dump()

머리노드의 레코드 : 3
0번째 레코드의 [data, next, dnext] : [40, -1, -1]
1번째 레코드의 [data, next, dnext] : [30, 0, -1]
2번째 레코드의 [data, next, dnext] : [20, 1, 4]
3번째 레코드의 [data, next, dnext] : [10, 1, -1]
4번째 레코드의 [data, next, dnext] : [50, -1, -1]
5번째 레코드의 [data, next, dnext] : [-1, -1, -1]
6번째 레코드의 [data, next, dnext] : [-1, -1, -1]
7번째 레코드의 [data, next, dnext] : [-1, -1, -1]
8번째 레코드의 [data, next, dnext] : [-1, -1, -1]
9번째 레코드의 [data, next, dnext] : [-1, -1, -1]


연결리스트의 current가 꼬리노드가 아닌 노드를 보고 있을 때,

    def remove() 함수로 current 노드를 삭제하게 되면 
    self.n[ptr].next = self.n[p].next 에 의해
    current노드의 직전노드 커서는 current 노드의 커서를 참조하게 된다. 
    다시 말해 잘 삭제된다.
    
    그리고 삭제뒤 새로운 current노드는 
    삭제 전 current의 직전노드가 된다

In [129]:
ALL.remove(ALL.current)
print(f'ALL.head : {ALL.head}')
print(f'ALL.current : {ALL.current}')
print(f'ALL.n[0].data : {ALL.n[0].data}')
print(f'ALL.n[0].next : {ALL.n[0].next}')
print(f'ALL.n[1].data : {ALL.n[1].data}')
print(f'ALL.n[1].next : {ALL.n[1].next}')
print(f'ALL.n[2].data : {ALL.n[2].data}')
print(f'ALL.n[2].next : {ALL.n[2].next}')
print(f'ALL.n[3].data : {ALL.n[3].data}')
print(f'ALL.n[3].next : {ALL.n[3].next}')
print(f'ALL.n[4].data : {ALL.n[4].data}')
print(f'ALL.n[4].next : {ALL.n[4].next}')

ALL.head : 1
ALL.current : 1
ALL.n[0].data : 40
ALL.n[0].next : -1
ALL.n[1].data : 30
ALL.n[1].next : 0
ALL.n[2].data : 20
ALL.n[2].next : 1
ALL.n[3].data : 10
ALL.n[3].next : 1
ALL.n[4].data : 50
ALL.n[4].next : -1


In [130]:
print(f'프리리스트 머리노드 ALL.deleted : {ALL.deleted}')
print(f'프리리스트 머리노드의 커서 ALL.n[ALL.deleted].dnext : {ALL.n[ALL.deleted].dnext}')
print(f'프리리스트 머리노드의 커서의 커서 ALL.n[ALL.n[ALL.deleted].dnext].dnext : {ALL.n[ALL.n[ALL.deleted].dnext].dnext}')

프리리스트 머리노드 ALL.deleted : 3
프리리스트 머리노드의 커서 ALL.n[ALL.deleted].dnext : 2
프리리스트 머리노드의 커서의 커서 ALL.n[ALL.n[ALL.deleted].dnext].dnext : 4


In [131]:
ALL.dump()

머리노드의 레코드 : 1
0번째 레코드의 [data, next, dnext] : [40, -1, -1]
1번째 레코드의 [data, next, dnext] : [30, 0, -1]
2번째 레코드의 [data, next, dnext] : [20, 1, 4]
3번째 레코드의 [data, next, dnext] : [10, 1, 2]
4번째 레코드의 [data, next, dnext] : [50, -1, -1]
5번째 레코드의 [data, next, dnext] : [-1, -1, -1]
6번째 레코드의 [data, next, dnext] : [-1, -1, -1]
7번째 레코드의 [data, next, dnext] : [-1, -1, -1]
8번째 레코드의 [data, next, dnext] : [-1, -1, -1]
9번째 레코드의 [data, next, dnext] : [-1, -1, -1]


In [132]:
for n in ALL:
    print(n)

30
40


# 문12) 파이썬의 논리연산자 and, or, not이 리턴하는 값
파이썬의 논리 연산자 and, or는 True, False 를 반환하지 않는다.

In [114]:
0 and 2 # and는 왼쪽 오른쪽 순서로 값을 평가해서 거짓이면 그값을 리턴, and는 왼쪽이 거짓이면 오른쪽 평가 생략

0

In [115]:
5 and 0 

0

In [112]:
5 and 3 # 둘다 참이면 오른쪽 값을 리턴

0

In [116]:
0 or 3 # or는 왼쪽 오른쪽 순서로 값을 평가해서 참이면 그값을 리턴,

3

In [51]:
1 or 0 # or는 왼쪽이 참이면 오른쪽 평가 생략

1

In [134]:
False or False

False

In [135]:
0 or 0

0

not은 True, False를 반환한다.  


# 문13) 논리 연산자 and, or, not의 우선순위
not, and, or 순이다.

In [53]:
not 0 and 1

1

In [54]:
not 1 and 0

False

In [55]:
not 0 or 0

True

In [58]:
not 1 or 1 and 2

2